# _Machine Learning_ e Automatização a partir de Imagens

Este _notebook_ contém uma demonstração de aplicação de _Machine Learning_ na deteção de anomalias a partir de imagens.

## Objectivos
Esta demonstação usa uma coleção de fotografias de sete bananas e tenciona treinar um modelo básico de deteção de anomalias visuais em frutos. Isto é feito em duas etapas:
1. Identificação do fruto contra o fundo da imagem, usando um método de redução da dimnesionalidade.
2. Identificação de anomalias em regiões da imangen identificadas como contendo o fruto, usando um método de aprendizagem não supervisionada.

## Resumo da Metodologia
As fotos usadas nesta demonstração têm todas uma dimensão de 1920 por 1080 pixeis (aproximadamente 2 MPixeis) de uma banana contra um fundo branco, e uma única fonte de iluminação colocada à frente e acima de máquina. Com excepção da banana nº 1 (a qual só tem duas fotografias: A e B), existem tês fotografias (A, B, e C) por banana. As fotografias das bananas 1 e 2 serão reservadas para aplicação do modelo final. Cada uma das outras fotos é processada da seguinte forma:
1. São recolhidas `a_nummber` alícotas de cada imagem, cada uma com 64x64 pixeis, em posições aleatórias,e gravadas numa tabela.
2. As alícotas das fotografias A e B de cada uma das bananas 3 a 7 são usadas para "treinar" um modelo de Análise de Componentes Principais.
3. As alícotas das fotografias C das bananas 3 a 7 são então usadas para aferir a capacidade do modelo PCA de identificar a banana contra o fundo, em imagens novas.
4. Cada fotografia A e B das bananas 3 a 7 é então "varrida" sistematicamente, de forma a identificar as regiões que retratam a banana em cada imagem. A localização dessas regiões é então gravada numa nova tabela.
5. As regiões encontradas no ponto anterior são usadas para treinar um modelo de _Support Vector Machine_ de forma a identificar anomalias visuais.
6. Os dois modelos são usados de forma conjugada para identificar a percentage de anomalias visuais detetadas em cada banana.

# Parâmetros da Demonstração

A célula abaixo define parâmetros que serão usados como constantes ao longo do trabalho.

In [ ]:
# tamanho de cada alicota (64 por 64 pixeis)
a_size = (64,64)
# numero de alicotas por imagem
a_number = 50

# pasta com as fotos
src_dir='./fotos/'

# estas fotos são reservadas para a demonstração final do modelo
reserve_files=['banana_1_A.png','banana_1_B.png',
               'banana_2_A.png','banana_2_B.png','banana_2_C.png']

## Bibliotecas de Funções Auxiliares
A célula abaixo carrega as bibliotecas e funções necessárias para o trabalho prático.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import image
%matplotlib inline

# criar um gerador de números aleatórios para a amostragem
my_rng = np.random.default_rng(seed=20250521)

def amostragem_aleatoria(fn):
    """Gera as posições de a_number alicotas na imagen do ficheiro fn e retorna
    uma lista de dicionarios (um por alicota) com nome do ficheiro, identificacao
    da banana e da fotografia, URI da foto e posicao (x e y) da alicota."""
    output = list()
    img = image.imread(fn)
    l=fn.name[:-4].split('_')[1:]
    # calculate the max values of x and y to draw an aliquot
    max_x = img.shape[1] - a_size[0]
    max_y = img.shape[0] - a_size[1]
    for n in range(a_number):
        o={}
        o['filename']=fn.name
        o['uri']=fn.absolute().as_uri()
        o['banana']=l[0]
        o['fotografia']=l[1]
        o['alicota']=n+1
        o['x']=my_rng.integers(max_x-1)
        o['y']=my_rng.integers(max_y-1)
        output.append(o)
    return output

def amostragem_em_grelha(fn):
    """Divide a imagem em fn numa grelha de alicotas e retorna uma lista de dicionarios
    (um por alicota) com nome do ficheiro, identificacao
    da banana e da fotografia, URI da foto e posicao (x e y) da alicota."""
    output = list()
    img = image.imread(fn)
    l=fn.name[:-4].split('_')[1:]
    # calculate the max values of x and y to draw an aliquot
    n = 0
    for x in np.arange(img.shape[1],step=a_size[0]):
        if x+a_size[0] < img.shape[1]:
            for y in np.arange(img.shape[0],step=a_size[1]):
                if y+a_size[1] < img.shape[0]:
                    o={}
                    o['filename']=fn.name
                    o['uri']=fn.absolute().as_uri()
                    o['banana']=l[0]
                    o['fotografia']=l[1]
                    o['alicota']=n+1
                    o['x']=x
                    o['y']=y
                    output.append(o)
                    n += 1
    return output

def normalizar_imagem(m):
    m -= m.min()
    m /= m.max()
    return m

def mostrar_imagem(fn, normalizar=False):
    f_img=image.imread(fn)
    if normalizar:
        f_img = normalizar_imagem(f_img)
    plt.matshow(f_img)

def mostrar_alicotas(a_list, banana, foto):
    have_img = False
    for a in a_list:
        if (int(a['banana'])==banana) and (a['fotografia']==foto.upper()):
            if not have_img:
                img = image.imread(Path.from_uri(a['uri']))
                img = normalizar_imagem(img)
                mask = np.ones(img.shape)
                have_img=True
            x=a['x']
            y=a['y']
            mask[y:y+a_size[1],x:x+a_size[0],:] *= 0.9
    plt.matshow(normalizar_imagem(mask*img))

def criar_X(a_list):
    """Cria uma array com as alicotas lidas a partir de a_list"""
    l=[]
    for a in a_list:
        img=image.imread(Path.from_uri(a['uri']))
        x = a['x']
        y = a['y']
        l.append(img[y:y+a_size[1],x:x+a_size[0],:].reshape(-1))
    return np.array(l)

def aplicar_modelo(modelo, a_list, X=None):
    """Aplica o modelo a uma lista de alicotas e devolve uma pandas.DataFrame
    com a resposta do modelo na coluna RESP"""
    if X is None:
        X = criar_X(a_list)
    if 'predict' in dir(modelo):
        resp = modelo.predict(X)
    elif 'transform' in dir(modelo):
        resp = modelo.transform(X)[:,0]
    else:
        raise NotImplementedError("Modelo desconhecido!")
    o=pd.DataFrame(a_list)
    o['RESP']=resp
    return o

def mostrar_resposta(r_df):
    """Lê uma DataFrame com a resposta de um modelo (coluna RESP) e 
    cria uma representação visual"""
    has_img = False
    for a in r_df.iterrows():
        if not has_img:
            img = image.imread(Path.from_uri(a[1]['uri']))
            mask = np.zeros(img.shape)
            has_img=True
        x=a[1]['x']
        y=a[1]['y']
        mask[y:y+a_size[1],x:x+a_size[0],0] += a[1]['RESP']
    plt.matshow(normalizar_imagem(mask))
    plt.matshow(normalizar_imagem(img*mask))


## Parte 1: Recolha das Alícotas

In [ ]:
alicotas = list()

for banana in range(3,8):
    for foto in ['A','B']:
        fn= Path(src_dir) / f"banana_{banana}_{foto}.png"
        alicotas += amostragem_aleatoria(fn)

print(f"Foram recolhidas {len(alicotas)} alicotas!")

In [ ]:
# mostrar as alicotas recolhidas da fotografia 'B' da banana no 4
mostrar_alicotas(alicotas,4,'B')

## Parte 2: Detetar Bananas com PCA

In [ ]:
from sklearn.decomposition import PCA

X = criar_X(alicotas)
m_pca = PCA(n_components=3)
m_pca.fit(X)
print(m_pca.explained_variance_ratio_)

In [ ]:
r_pca = aplicar_modelo(m_pca,alicotas,X)

In [ ]:
mostrar_resposta(r_pca[r_pca['filename']=='banana_4_A.png'])

In [ ]:
grelha = amostragem_em_grelha(Path(src_dir)/'banana_3_C.png')
print(f"A grelha contém {len(grelha)} alicotas.")
r_grelha = aplicar_modelo(m_pca,grelha)
mostrar_resposta(r_grelha)

## Parte 2: Deteção de Anomalias Visuais em Bananas

Em primeiro lugar, vamos aplicar o modelo PCA a uma amostragem de grelha de todas as fotografias A e B das bananas 3 a 7, e selecionar apenas as alícotas com PC1 < 0 (no exemplo acima as bananas são manchadas a ciano, o que implica que o canal vermelho foi reduzido pela máscara).

In [ ]:
alicotas2 = list()

for banana in range(3,8):
    for foto in ['A','B']:
        fn= Path(src_dir) / f"banana_{banana}_{foto}.png"
        alicotas2 += amostragem_aleatoria(fn)

print(f"Foram recolhidas {len(alicotas2)} alicotas!")

A próxima célula pode demorar um tempo considerável a correr.

In [ ]:
resultado = aplicar_modelo(m_pca,alicotas2)

In [ ]:
filtrado = resultado[resultado['RESP']<0]
# remover a coluna 'RESP' e converter numa lista de dicionarios para aplicar novo modelo
bananas = filtrado.drop(columns=['RESP']).to_dict('records')
print(f"Temos {len(bananas)} alicotas de banana! :)")

Agora, criamos um modelo de Isolation Forest para detacção de anomalias. Mais uma vez, a célula seguinte pode demorar algum tempo a correr.

In [ ]:
#from sklearn.ensemble import IsolationForest

#m_if = IsolationForest()
X=criar_X(bananas)
#m_if.fit(X)

from sklearn.svm import OneClassSVM
m_svm = OneClassSVM()
m_svm.fit(X)

In [ ]:
#r_if = aplicar_modelo(m_if,bananas,X)
r_svm = aplicar_modelo(m_svm,bananas,X)

In [ ]:
mostrar_resposta(r_svm[r_svm['filename']=='banana_7_A.png'])

## Parte 3: Aplicação dos modelos.

Finalmente, vamos aplicar os modelos a duas bananas "novas": banana 1 e 2. para isso defiimos uma nova função que condensa a análise e toma como entrada apenas o nome do ficheiro, e devolve, para além das imagens, a perentagem de banana classificada pelo SVM como parte da classe "+1".

In [ ]:
ficheiro=Path(src_dir) / 'banana_1_A.png'
def analisar_banana(ficheiro):
    grelha1=amostragem_em_grelha(ficheiro)
    print(f"A grelha tem {len(grelha1)} alicotas")
    r1=aplicar_modelo(m_pca,grelha1,criar_X(grelha1))
    mostrar_resposta(r1)
    f1 = r1[r1['RESP']<0]
    # remover a coluna 'RESP' e converter numa lista de dicionarios para aplicar novo modelo
    b1 = f1.drop(columns=['RESP']).to_dict('records')
    print(f"Temos {len(b1)} alicotas de banana! :)")
    r2 = aplicar_modelo(m_svm,b1,criar_X(b1))
    mostrar_resposta(r2)
    return 100.0*(len(r2[r2['RESP'] == 1])/len(b1))


In [ ]:
analisar_banana(Path(src_dir) / 'banana_1_A.png')

In [ ]:
analisar_banana(Path(src_dir) / 'banana_1_B.png')

In [ ]:
analisar_banana(Path(src_dir) / 'banana_2_A.png')

In [ ]:
analisar_banana(Path(src_dir) / 'banana_2_B.png')